In [1]:
// Nuget Packages
#r "nuget: MelonChart.NET, 2.*"

#r "nuget: Microsoft.Extensions.Configuration.Json, 8.*"
#r "nuget: Microsoft.SemanticKernel, 1.*"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.*"
#r "nuget: Microsoft.SemanticKernel.Core, 1.*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.*-*"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.*-*"

#r "nuget: System.Linq.Async, 6.*"


Installed Packages MelonChart.NET, 2.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Microsoft.SemanticKernel, 1.16.0 Microsoft.SemanticKernel.Connectors.OpenAI, 1.16.0 Microsoft.SemanticKernel.Core, 1.16.0 Microsoft.SemanticKernel.Plugins.Core, 1.16.0-alpha Microsoft.SemanticKernel.Plugins.Memory, 1.16.0-alpha System.Linq.Async, 6.0.1

In [2]:
// Add using statements
using System.ComponentModel;
using System.IO;
using System.Net.Http;
using System.Text.Encodings.Web;
using System.Text.Json;
using System.Text.Json.Serialization;

using MelonChart.Models;

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Memory;

using Kernel = Microsoft.SemanticKernel.Kernel;

In [3]:
// Azure OpenAI configurations
var path = Path.Combine(Directory.GetCurrentDirectory(), "appsettings.Development.json");
var config = new ConfigurationBuilder().AddJsonFile(path).Build();

var endpoint = config["Azure:OpenAI:Endpoint"];
var apiKey = config["Azure:OpenAI:ApiKey"];
var deploymentName = config["Azure:OpenAI:DeploymentName"];

In [20]:
// Build Semantic Kernel
var kernel = Kernel.CreateBuilder()
                   .AddAzureOpenAIChatCompletion(
                       deploymentName: deploymentName,
                       endpoint: endpoint,
                       apiKey: apiKey)
                   .Build();

In [21]:
// Import prompts
var prompts = kernel.ImportPluginFromPromptDirectory("Prompts");

In [22]:
// User input
var question = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("지금 무엇을 도와드릴까요?");

Console.WriteLine($"User: {question}");

User: 날이 좋아서 무엇이든 좋았다


In [23]:
// Invoke the prompt - Get Sentiments
var sentiments = await kernel.InvokeAsync<string>(
                       function: prompts["GetSentiments"],
                       arguments: new KernelArguments()
                       {
                           { "input", question }
                       });

Console.WriteLine(sentiments);

1. Contentment
2. Serenity
3. Optimism
4. Gratitude
5. Peacefulness


In [24]:
// Invoke the prompt - Measure Sentiments
var measured = await kernel.InvokeAsync<string>(
                     function: prompts["MeasureSentiments"],
                     arguments: new KernelArguments()
                     {
                         { "input", question },
                         { "sentiments", sentiments },
                     });

Console.WriteLine(measured);

| Sentiment    | Danceability | Valence     |
| -------------|--------------|-------------|
| Contentment  | 0.50 - 0.70  | 0.60 - 0.80 |
| Serenity     | 0.40 - 0.60  | 0.60 - 0.80 |
| Optimism     | 0.60 - 0.80  | 0.70 - 0.90 |
| Gratitude    | 0.50 - 0.70  | 0.70 - 0.90 |
| Peacefulness | 0.40 - 0.60  | 0.60 - 0.80 |


In [25]:
// Invoke the prompt - Pick Primary Sentiment
var sentiment = await kernel.InvokeAsync<string>(
                     function: prompts["PickPrimarySentiment"],
                     arguments: new KernelArguments()
                     {
                         { "input", question },
                         { "measured", measured },
                     });

Console.WriteLine(sentiment);

```json
{
  "sentiment": "Contentment",
  "danceability": "0.50 - 0.70",
  "valence": "0.60 - 0.80"
}
```
